In [1]:
pip install networkx

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import networkx as nx

In [3]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [4]:
data = pd.read_csv('full_data2.csv')

/Users/sjk252/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (5,39,49) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
data.head()

,index,Provider,PotentialFraud,BeneID,DOB,DOD,Gender,Race,RenalDiseaseIndicator,State,County,NoOfMonths_PartACov,NoOfMonths_PartBCov,ChronicCond_Alzheimer,ChronicCond_Heartfailure,ChronicCond_KidneyDisease,ChronicCond_Cancer,ChronicCond_ObstrPulmonary,ChronicCond_Depression,ChronicCond_Diabetes,ChronicCond_IschemicHeart,ChronicCond_Osteoporasis,ChronicCond_rheumatoidarthritis,ChronicCond_stroke,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,ClaimID,ClaimStartDt,ClaimEndDt,InscClaimAmtReimbursed,AttendingPhysician,OperatingPhysician,OtherPhysician,AdmissionDt,ClmAdmitDiagnosisCode,DeductibleAmtPaid,DischargeDt,DiagnosisGroupCode,ClmDiagnosisCode_1,ClmDiagnosisCode_2,ClmDiagnosisCode_3,ClmDiagnosisCode_4,ClmDiagnosisCode_5,ClmDiagnosisCode_6,ClmDiagnosisCode_7,ClmDiagnosisCode_8,ClmDiagnosisCode_9,ClmDiagnosisCode_10,ClmProcedureCode_1,ClmProcedureCode_2,ClmProcedureCode_3,ClmProcedureCode_4,ClmProcedureCode_5,ClmProcedureCode_6,patient_type,Physician,prov_by_AttPhys,prov_by_patient,patient_visits,age,num_chronic,total_reimbursement,ReimburseByVisit,total_deductible,DeductibleByVisit,ins_cov_perc,claim_days,TotClaimAmt,mean_daily_charge,year,week,treatment_days,month,duplicate,ReceiveInfo,ProvideInfo
0,412814,PRV55832,No,BENE109092,1926-03-01,NaN,2,1,0,38,320,12,12,1,1,0,0,1,1,1,1,1,0,0,16830,6204,1120,820,CLM31001,2008-11-27,2009-01-01,10000,PHY402579,NaN,PHY402579,2008-11-27,29570,1068.0,2009-01-01,876,29534,49120,27800,3019,4019,07070,25000,2724,V142,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,1.0,3,6,82,7,17950,2991.666667,7024,1170.666667,0.903506,35.0,11068.0,316.228571,2008,48,35.0,11,0,0,0
1,418693,PRV55897,Yes,BENE136624,1937-07-01,NaN,2,1,1,39,680,12,12,0,1,1,0,1,0,1,1,0,0,0,108000,3204,22550,5650,CLM31004,2008-11-27,2009-01-01,51000,PHY407408,PHY366859,NaN,2008-11-27,70700,1068.0,2009-01-01,941,V5875,5601,43882,53190,5849,70723,40390,V1254,41400,NaN,9904.0,3843.0,NaN,NaN,NaN,NaN,1,1,1.0,6,18,71,5,130550,7252.777778,8854,491.888889,0.979488,35.0,52068.0,1487.657143,2008,48,35.0,11,0,0,0
2,481250,PRV56588,Yes,BENE82606,1922-09-01,NaN,2,1,0,45,390,12,12,1,1,0,1,1,1,1,0,0,1,0,28240,2136,290,390,CLM31006,2008-11-28,2009-01-02,18000,PHY356371,PHY415979,NaN,2008-11-28,7866,1068.0,2009-01-02,196,1625,2859,43310,42731,2724,53081,496,5128,2761,NaN,3249.0,NaN,NaN,NaN,NaN,NaN,1,1,1.0,4,6,86,7,28530,4755.000000,2526,421.000000,0.943990,35.0,19068.0,544.800000,2008,48,35.0,11,0,0,0
3,37428,PRV51456,No,BENE151770,1915-07-01,NaN,2,1,0,5,140,12,12,0,0,0,0,0,0,0,1,0,0,0,59600,1068,580,160,CLM31005,2008-11-28,2009-01-02,56000,PHY428454,PHY428454,NaN,2008-11-28,4240,1068.0,2009-01-02,309,41401,4240,30000,9971,E8781,5185,45829,2851,4019,NaN,3521.0,5185.0,NaN,NaN,NaN,NaN,1,1,2.0,3,4,93,1,60180,15045.000000,1228,307.000000,0.981285,35.0,57068.0,1630.514286,2008,48,35.0,11,0,0,0
4,138224,PRV52396,Yes,BENE31729,1937-05-01,NaN,1,1,1,11,963,12,12,1,1,1,0,1,1,1,1,0,1,1,21000,3204,1150,880,CLM31007,2008-11-29,2009-01-03,0,PHY314706,NaN,NaN,2008-11-29,78650,1068.0,2009-01-03,OTH,2920,2912,7994,30401,V4364,4019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,1.0,2,2,71,9,22150,11075.000000,4084,2042.000000,0.000000,35.0,1068.0,30.514286,2008,48,35.0,11,0,0,0


In [6]:
def network_connections(data):
    """
    This function conducts social network analysis and returns two added features to the data set:
    1) Number of physicians connected to each provider
    2) Number of patients connected to each provider
    """
    Full_soc_dataProv = data.groupby(['Provider','AttendingPhysician'])['IPAnnualReimbursementAmt'].count().reset_index()
    Full_soc_dataPatient = data.groupby(['Provider','BeneID'])['IPAnnualReimbursementAmt'].count().reset_index()
    
    G1 = nx.Graph()
    G1 = nx.from_pandas_edgelist(Full_soc_dataProv, 'Provider','AttendingPhysician')
    prov_phys_degree = pd.DataFrame(G1.degree)
    prov_phys_degree.columns = ['Provider','AttPhys_Connections']
    prov_phys_degree2 = prov_phys_degree[prov_phys_degree['Provider'].str.contains('PRV')]
    data = pd.merge(prov_phys_degree2, data, how="outer", on="Provider")
    
    G2 = nx.Graph()
    G2 = nx.from_pandas_edgelist(Full_soc_dataPatient, 'Provider','BeneID')
    prov_patient_degree = pd.DataFrame(G2.degree)
    prov_patient_degree.columns = ['Provider','Patient_Connections']
    prov_patient_degree2 = prov_patient_degree[prov_patient_degree['Provider'].str.contains('PRV')]
    data = pd.merge(prov_patient_degree2, data, how="outer", on="Provider")
    
    return data

In [7]:
data = network_connections(data)

In [8]:
data.head()

,Provider,Patient_Connections,AttPhys_Connections,index,PotentialFraud,BeneID,DOB,DOD,Gender,Race,RenalDiseaseIndicator,State,County,NoOfMonths_PartACov,NoOfMonths_PartBCov,ChronicCond_Alzheimer,ChronicCond_Heartfailure,ChronicCond_KidneyDisease,ChronicCond_Cancer,ChronicCond_ObstrPulmonary,ChronicCond_Depression,ChronicCond_Diabetes,ChronicCond_IschemicHeart,ChronicCond_Osteoporasis,ChronicCond_rheumatoidarthritis,ChronicCond_stroke,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,ClaimID,ClaimStartDt,ClaimEndDt,InscClaimAmtReimbursed,AttendingPhysician,OperatingPhysician,OtherPhysician,AdmissionDt,ClmAdmitDiagnosisCode,DeductibleAmtPaid,DischargeDt,DiagnosisGroupCode,ClmDiagnosisCode_1,ClmDiagnosisCode_2,ClmDiagnosisCode_3,ClmDiagnosisCode_4,ClmDiagnosisCode_5,ClmDiagnosisCode_6,ClmDiagnosisCode_7,ClmDiagnosisCode_8,ClmDiagnosisCode_9,ClmDiagnosisCode_10,ClmProcedureCode_1,ClmProcedureCode_2,ClmProcedureCode_3,ClmProcedureCode_4,ClmProcedureCode_5,ClmProcedureCode_6,patient_type,Physician,prov_by_AttPhys,prov_by_patient,patient_visits,age,num_chronic,total_reimbursement,ReimburseByVisit,total_deductible,DeductibleByVisit,ins_cov_perc,claim_days,TotClaimAmt,mean_daily_charge,year,week,treatment_days,month,duplicate,ReceiveInfo,ProvideInfo
0,PRV51001,24,14,3,No,BENE86784,1934-01-01,NaN,2,1,0,1,440,12,12,1,0,0,0,0,0,1,1,1,0,0,0,0,2650,700,CLM119495,2009-01-04,2009-01-04,10,PHY350459,NaN,PHY350459,NaN,NaN,0.0,NaN,NaN,2767,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,1,2.0,2,5,75,4,2650,530.0,700,140.0,1.000000,0.0,10.0,0.0,2009,1,NaN,1,1,0,1
1,PRV51001,24,14,9,No,BENE130029,1927-03-01,NaN,2,1,0,1,660,12,12,1,1,0,0,1,0,1,1,0,0,1,7000,1068,4480,200,CLM168357,2009-01-31,2009-01-31,1000,PHY350459,NaN,PHY350459,NaN,NaN,0.0,NaN,NaN,4110,V4581,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,1,2.0,4,5,81,6,11480,2296.0,1268,253.6,1.000000,0.0,1000.0,0.0,2009,5,NaN,1,0,0,0
2,PRV51001,24,14,18,No,BENE123416,1927-03-01,NaN,2,1,0,1,440,12,12,0,0,1,0,0,0,1,1,0,0,0,0,0,3600,10,CLM172987,2009-02-02,2009-02-02,100,PHY345302,NaN,PHY345302,NaN,NaN,0.0,NaN,NaN,24900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,1,1.0,4,4,81,3,3600,900.0,10,2.5,1.000000,0.0,100.0,0.0,2009,6,NaN,2,1,1,0
3,PRV51001,24,14,24,No,BENE98831,1934-11-01,NaN,2,1,0,1,510,12,12,1,1,0,0,0,1,1,1,0,1,0,3000,1068,0,0,CLM36844,2009-02-03,2009-02-08,3000,PHY383193,NaN,NaN,2009-02-03,5362,1068.0,2009-02-08,353,56211,5533,0088,5789,53081,27651,7840,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,1.0,1,1,74,6,3000,3000.0,1068,1068.0,0.737463,5.0,4068.0,813.6,2009,6,5.0,2,0,0,0
4,PRV51001,24,14,20,No,BENE152088,1913-09-01,NaN,2,1,1,1,160,12,12,1,1,1,1,1,1,1,1,0,1,1,111710,4272,80,70,CLM37205,2009-02-06,2009-02-06,4000,PHY402229,PHY329739,NaN,2009-02-06,80121,1068.0,2009-02-06,062,85222,2724,3310,40390,V4501,V5866,NaN,NaN,NaN,NaN,8659.0,NaN,NaN,NaN,NaN,NaN,1,1,1.0,3,5,95,10,111790,22358.0,4342,868.4,0.789266,0.0,5068.0,0.0,2009,6,0.0,2,0,0,0
